
---
<big><big><big><big><big><big>Sieci neuronowe 2018</big></big></big></big></big></big>

---





---
<big><big><big><big><big>Głębokie sieci warstwowe</big></big></big></big></big>

---



---

<id=tocheading><big><big><big><big>Spis treści</big></big></big></big>
<div id="toc"></div>

---

In [4]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [3]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

#from bokeh.io import gridplot, output_file, show
#from bokeh.plotting import figure, output_notebook
#from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Głębokie sieci warstwowe
1. w jaki sposób nauczyć pojedyncze neurony?
  * bardzo podstawowe modele nieskończenie wymiarowe
    * wykorzystywane przez sieci RBF
    * jesli $\phi(x)$ będzie wystarczająco wysoko wymiarowe, to dopasuje się do zbioru uczącego
    * generalizacja słaba
  * wyliczyć je ręcznie
  * wykorzystać modele __głębokie__ i nauczyć się $\phi()$
    * typowo używamy funckji aktywacji _rectified linear_ __ReLU__$(x)=max(0, x)$


## Funkcje kosztu
1. __maximum likelihood__
$$L(w)=\mathbb{E}_{x,y\sim\widehat{p}_{DATA}}\log\,p_{model}(y\mid x)$$
  * jeśli zakładmy konkretny rozkład prawdopodobieństwa warunkowego, to dostaniemy konkretną funkcję kosztu
  * dla $p_{model}(y\mid x)=\mathcal{N}(y; f(x;w), \mathbf{I})$ dostaniemy 
  $$L(w)=\frac{1}{2}\mathbb{E}_{x,y\sim\widehat{p}_{DATA}}\|y-f(x;w\|^2+const$$

## Neurony wyjściowe
### __liniowe__ $y=w^Th+b$
  * dla średniej warunkowego rozkładu gausowskiego $$p(y\mid x)\mathcal{N}(y; f(x;w), \mathbf{I})$$
    * można też przewidywać macierz kowariancji $\Sigma$
      * jednak musi być dodatnio określona
      * zwykle ogranicza się do $\Sigma=\sigma\mathbb{I}$
      
### __sigmoidalne__ dla rozkładu Bernoulliego $y=\sigma(W^Th+b)$
  * nigdy nie osiąga wartosci granicznych
  * użycie liniowego obiętego do przedziału $[0,1]$ jest trudne w uczeniu
    * po przejściu przez wartości graniczne gradient byłby zerowy

### __softmax__ dla wielowymiarowego rozkładu Bernoulliego
  * dla zmiennych dyskretnych o $n$ możliwych wartościach
    * warstwa liniowa $z=w^Th+b$
    gdzie $z_i=\log\,P(y=i\mid x)$
    * aby znaleźć $\widehat{y}$ $$softmax\,(z)_i=\frac{\exp(z_i)}{\sum_j\exp(z_j)}$$
    * będzie działac prawidłowo przy uczeniu przez maximum log-likelihood
    $$\log\,softmax(z)_i=z_i-\log\,\sum_j\exp(z_j)$$
      * pierwszy składnik będzie pchany do góry, pozostałe w dół
    * drugi składnik można aproksymować 
    $$\log\,\sum_j\exp(z_j)\simeq\max_j\,z_j$$
      * najsilniej będzie karana największa z niepoprawnych odpowiedzi
      * jeśli największą wartość ma już $z_i$, to
      $$\log\,\sum_j\exp(z_j)\simeq\max_j\,z_j=z_i$$
      i oba składniki będą się praktycznie zrowały
        * przykład nie bedzie miał wielkiego wpływu na uczenie
  * może być użyty dla neuronów ukrytych
    * by neuron wybierał jedną z wielu wartości
  * funkcje celu różne od _log-likelihood_ __nie będą__ dobrze współpracowały z _softmax_
    * w szczególności będzie nią błąd kwadratowy
  * ustabilizowana wersja softmax
    * dla dużych wartości $z$, softmax bedzie się wypłaszczał bardzo
    * _softmax_ jest odporny na translację_
    * można uniezależnić od ekstremalnych wartosci przez
    $$softmax(z)=softmax(z-\max_j\,z_j)$$
    
### __gausowskie__
* może reprezentować warunkowy rozkład gausowski dla $y$ pod warunkiem $x$
  * dla ustalonego $\sigma$
  * dla uczonej macierzy kowariancji $\Sigma$
    * rzadko kiedy innej niż diagonalna

## Neurony __ukryte__
* nigdy nie ma całkowitej pewnosci neurony którego typu będą działały najlepiej
* nie wszystkie typy są __wszędzie__ różniczkowalne
  * __ReLU__ $\max(0,x)$ nie jest w $0$
  * nie oczekujemy, że sieć rzeczywiscie dotrze do punktu o gradiencie $0$
    * stąd punkty o nieokreślonym gradiencie są dopuszczalne
    * jeśli pochodne lewa i prawa są różne, to oprogramowanie zwykle zwraca jedną z nich
    * rzeczywista wartość zwykle i tak różni się o $\epsilon$
  * 

### Rectified Linear Unit __ReLU__
1. aktywacja $$g(z)=\max(0, z)$$
  * pochodna dodatnia kiedykolwiek neuron jest aktywny
  * druga pochodna prawie wszędzie jest zerowa
2. neurony __nie uczą__ się w miejscach, gdzie nie są aktywne
  * niezerowe nchylenie dla $z_i<0$
  $$g(z;\alpha)_i=max(0,z_i)+\alpha_i\min(0, z_i)$$
  * do wykorzystania, gdy cechy są inwariantne na odwrócenie odczytu, np. całkowita zmiana oświetlenie
    * __stałe__ $\alpha_i$ w __Leaky ReLU__
    * __parametryczne__ $\alpha_i$ gdy $\alpha_i$ jest douczane
3. __maxout__ 
  * złożenie wielu wartości liniowych $z_i$
  $$g(z)_i=\max_{j\in{}Ind(i)}z_j$$
  * maxout uczy się odcinkowo liniowej wypukłej funkcji
4. wybór przez założenie o prostocie
  * łatwiej uczyć modele, jesli są bardziej podobne do liniowych

### Neurony ukryte z sigmoidalnymi funkcjami aktywacji
1. __logistyczna $g(z)=\sigma(z)$
2. __tanh__ $$g(z)=\tanh(z)=2\sigma(z)-1$$
  * saturuje podobnie jak logistyczna
  * w okolicach zera uczenie przypomina uczenie sieci liniowych
  

### Inne neurony ukryte
1. jest bardzo wiele funkcji aktywacji
2. __funkcja liniowa__
  * zwykle tylko nietóre
3. __softmax__
  * rzadko, ale mogą wymusić pewne wybory
4. __radialne__
5. __softplus__
$$g(z)=\log(1+\exp(z))$$
  * wydaje się wygładzoną wersją ReLU
  * doświadczalnie sprawuje się słabiej
5. __ograniczont $\tnh$__
$$g(z)=\max(-1,\min(1,z))$$

# Głębokie sieci
> __Ogólne twierdzenie o aproksymacji__ mówi, że sieć warstwowa, z liniowymi neuronami ukrytymi oraz jedną warstwą ukrytą ze _zgniatającą_ funkcją aktywacji (np. logistyczną) jest w stanie aproksymować z dowolną dokładnością dowolną mierzalną funkcję ciągłą na domknietym podzbiorze $\mathbb{R}^n$ pod warunkiem, że sieć będzie miała __wystarczającą__ liczbe neuronów ukrytych.

* liczba neuronów ukrytych jest zwykle wykładniczo duża
* sieć będzie potrafiła __aproksymować__ funkcję, ale nie ma żadnej gwarancji, że się jej __nauczy__
* ale istnieje rodzina funkcji, które mogą być przedstawione pod warunkiem odpowiedniej __głębokości__
* empirycznie wiemy, że większa głębokość daje zwykle lepszą generalizację

# Regularyzacja
Celem regularyzacji jest osiągniecie modelu o niższej złożoności.
* zwykle celem jest minimalizacja __wariancji__ kosztem nieco zwiększonego __biasu__

## Regularyzacja przez czynnik kary
$$L(w; X, y)=L(w; X, y)+\alpha\Omega(w)$$
* algorytm uczący bedzie minimalizował koszt na danych
* oraz jakąś miarę na parametrach $w$
  * __nie__ regularyzujemy biasów
  * wagi kontrolują interakcję między dwoma neuronami
  * biasy tylko jednego neuronu
  * biasy wprowadzają mniej wariancji w modelu
* może się okazać potrzebne __oddzielne__ regularyzowanie różnych warstw
  * różne $\alpha$ dla róznych warstw

## $L2$
$$\Omega(w)=\frac{1}{2}\|w\|_2^2$$
* ridge regression albo regresja Tikhonova
$$\begin{align}
w&=w-\epsilon(\alpha{}w+\nabla_wL(w))\\
&=(1-\epsilon\alpha)w-\epsilon\nabla_wL(w)
\end{align}$$

Regularyzacja L2 powoduje __skalowanie__ optymalnego wektora wag $w^*$ __wzdłuż__ wektorów głównych zdefiniowanych przez macierz Hesjanu $H$
<img src="nn_figures/L2.pdf" width="100%"> [Goodfellow et al.]
1. wektor $w*$ jest przeskalowany wzdłuż $i$-tego wektora głównego $H$ przez czynnik $\dfrac{\lambda_i}{\lambda_i+\alpha}$, gdzie $\alpha$ jest czynnikiem  funkcji kary
  * jeśli $\lambda_i>>\alpha$, to regularyzacja będzie niewielka
  * jeśli $\lambda_i<<\alpha$, to współrzędna będzie zredukowana prawie do minimum
2. na rysunku
  * $\tilde{\ w}$ jest punktem ekwilibrium miedzy minimum kosztu, a regularyzacją
  * w pierwszym głównym kierunku wzdłuż $w_1$ wartość własna $H$ jest niewielka
    * funkcja kosztu nie zmienia się wiele w tym kierunku, a wobec tego regularyzator ma duży wpływ i skraca $w_1$ do zera
  * w drugim kierunku funkcja celu jest bardzo czula na zmiany
    * wartość własna jest wysoka
    * regularyzator mało wpływa

## $L1$
Składnik kosztu jako $$\Omega(w)=\|w\|_1=\sum_i\mid w_i\mid$$
* tylko wagi, nie bias
* gradient $$\nabla_wL(w)=\nabla_wL(w)+\alpha{}sign(w)$$
* zachowanie jest inne niż $L2$
  * gradient nie skaluje się liniowo z każdym $w$, stąd brak czystych rozwiązań
  * rozwinięcie funkcji kosztu wraz z poprawką $L1$ 
  $$L(w)=L(w+(w-w^*))=L(w)+\sum_i\left[\frac{1}{2}H(w-w^*)+\alpha\mid{}w_i\mid\right]$$
    * skladnik pierwszej pochodnej znika, bo $w^*$ jest optimum
  * rozwiązanie analityczne ma postać
  $$w_i=sign(w_i^*)\,\max\left[\mid{}w_i^*\mid-\frac{\alpha}{H_{i,i}},\,0\right]$$
  zakładając, że $H$ jest diagonalna
    * jeśli $\mid{}w_i^*\mid\leq\frac{\alpha}{H_{i,i}}$
      * tu rozwiązaniem jest $w_i=0$
      * regularyzacja przeważa
    * jeśli $\mid{}w_i^*\mid>\frac{\alpha}{H_{i,i}}$
      * $w_i$ jest przesuwane
* $L1$ daje bardziej __rzadkie__ rozwiązanie


## Rozszerzanie zbioru uczącego
1. na pewno lepiej jest mieć więcej danych niż mniej
2. sztuczne dane
  * klasyfikator powinien być inwariantny na transformacje - wystarczy je zastosować
  * ale ostrożnie gdy mogą one zmienić znaczenie: 6 a 9
3. szum losowy
  * także dodawany do neuronów ukrytych
  * sieci bywają mało odporne na szum
  * wykorzystywane w tzw. __denoising autoencoders__
  * także __dropout__ jest formą dodawania szumu (właściwie __mnożenia__ przez szum)
4. ostrożność przy porównywaniu algorytmów
  * jeden algorytm działa słabo przy danych czystych
  * drugi dobrze przy danych rozszerzonych
  * jaki można wyciągnąć wniosek?
  * ale pamiętac by warunki porównania były sprawiedliwe (ten sam zbiór dla porównań)

## Dodawanie szumu do neuronów wyjsciowych
* zbiory mają błędy w etykietach
* maksymalizacja $\log\,p(y\mid x)$ może zaszkodzić, jesli etykieta $y$ jest błędna
* jak temu zaradzić?
  * modelować szum etykietowań
  * załóżmy, że etykiety są poprawne z prawdopodobieństwem $1-\epsilon$
  * najlepiej __dodać__ to założenie do funkcji kosztu
  * __label smoothing__
    * zastąpienie w modelu opartym o _softmax_ etykiet $0$ i $1$ przez $\frac{\epsilon}{k}$ i $1-\frac{k-\epsilon}{k}$
    * uczenie nigdy nie dotrze do etykiet $0$ i $1$
    * będzie się długo uczyć ciągle __powiekszając__ wagi
    * konieczne dodanie jakiegoś _weight decay_

## Uczenie semi-nadzorowane

## Uczenie wielo-zadaniowe